In [16]:
import pandas as pd
from huggingface_hub import hf_hub_download

REPO_ID = "phil329/OpenVid-1M-mapping"
dfs = []

# Loop through parts 0 to 185 (inclusive)
for part in range(186):
    filename = f"video_mappings/OpenVid_part{part}.csv"
    file_path = hf_hub_download(
        repo_id=REPO_ID,
        filename=filename,
        repo_type="dataset",
        local_dir="."
    )
    df = pd.read_csv(file_path)
    dfs.append(df)

# Combine all individual DataFrames into one
dataset = pd.concat(dfs, ignore_index=True)

# Optionally, inspect the resulting dataset
print(dataset.head())


            zip_file                     video  \
0  OpenVid_part0.zip  celebv_--Jiv5iYqT8_0.mp4   
1  OpenVid_part0.zip  celebv_--Jiv5iYqT8_2.mp4   
2  OpenVid_part0.zip  celebv_--QCZKgJt6o_0.mp4   
3  OpenVid_part0.zip  celebv_--oCWVOBuvA_0.mp4   
4  OpenVid_part0.zip  celebv_--uyzf7X_0c_0.mp4   

                               video_path  
0  OpenVid_part1/celebv_--Jiv5iYqT8_0.mp4  
1  OpenVid_part1/celebv_--Jiv5iYqT8_2.mp4  
2  OpenVid_part1/celebv_--QCZKgJt6o_0.mp4  
3  OpenVid_part1/celebv_--oCWVOBuvA_0.mp4  
4  OpenVid_part1/celebv_--uyzf7X_0c_0.mp4  
